# Baseline: Pretrained CLIP embeddings

In [5]:
import numpy
import pandas
import pathlib
import shutil
import seaborn as sns

from tqdm import tqdm
from PIL import Image
from collections import Counter
from transformers import CLIPTokenizer, CLIPTextModel
from transformers import CLIPProcessor, CLIPVisionModel

from typing import Dict, List, Optional, Union, Tuple

## Read Artemis

We use the "clean" version as specified by the authors' script.

In [6]:
artemis_path = pathlib.Path("../../artemis/CLEAN_ARTEMIS/artemis_preprocessed.csv")
artemis = pandas.read_csv(artemis_path) if artemis_path.exists() else IOError
artemis

,art_style,painting,emotion,utterance,repetition,split,tokens,tokens_len,utterance_spelled,tokens_encoded,emotion_label
0,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,something else,"She seems very happy in the picture, and you w...",10,train,"['she', 'seems', 'very', 'happy', 'in', 'the',...",18,she seems very happy in the picture and you wa...,"[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",8
1,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,sadness,This woman has really knotty hands which makes...,10,train,"['this', 'woman', 'has', 'really', 'knotty', '...",14,this woman has really knotty hands which makes...,"[1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30...",7
2,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,something else,"When looking at this woman, I am filled with c...",10,train,"['when', 'looking', 'at', 'this', 'woman', 'i'...",27,when looking at this woman i am filled with cu...,"[1, 31, 32, 33, 20, 21, 34, 35, 36, 37, 38, 39...",8
3,Post_Impressionism,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,contentment,"A woman looking at ease, peaceful, and satisfi...",10,train,"['a', 'woman', 'looking', 'at', 'ease', 'peace...",15,a woman looking at ease peaceful and satisfied...,"[1, 44, 21, 32, 33, 47, 48, 11, 49, 50, 28, 51...",2
4,Expressionism,wassily-kandinsky_study-for-autumn-1909,awe,The bright colors make a very unique scene for...,7,train,"['the', 'bright', 'colors', 'make', 'a', 'very...",12,the bright colors make a very unique scene for...,"[1, 9, 55, 56, 57, 44, 6, 58, 59, 60, 9, 61, 6...",1
...,...,...,...,...,...,...,...,...,...,...,...
429426,Mannerism_Late_Renaissance,el-greco_st-francis-and-brother-leo-meditating...,fear,The skull and the mans very skinny face,50,rest,"['the', 'skull', 'and', 'the', 'mans', 'very',...",8,the skull and the mans very skinny face,"[1, 9, 1995, 11, 9, 131, 6, 1598, 46, 2, 0, 0,...",6
429427,Art_Nouveau_Modern,tom-thomson_woodland-waterfall-1916,sadness,I feel sadness because of the waterfall flowin...,50,rest,"['i', 'feel', 'sadness', 'because', 'of', 'the...",20,i feel sadness because of the waterfall flowin...,"[1, 34, 53, 403, 288, 64, 9, 1876, 3596, 8, 44...",7
429428,Post_Impressionism,david-burliuk_landscape-1,awe,The greenery landscape and flowery background ...,50,rest,"['the', 'greenery', 'landscape', 'and', 'flowe...",10,the greenery landscape and flowery background ...,"[1, 9, 3910, 639, 11, 3065, 422, 64, 476, 17, ...",1
429429,Cubism,willi-baumeister_machine-man-with-spiral-turn-...,something else,The interlocking mechanical shapes fitting tog...,50,rest,"['the', 'interlocking', 'mechanical', 'shapes'...",10,the interlocking mechanical shapes fitting tog...,"[1, 9, 4096, 4101, 62, 3943, 696, 74, 3738, 11...",8


## Load precomputed embeddings

In [7]:
embd_folder = pathlib.Path(
    '/mnt/HDD/PROJECTS/EmotionalArt/CLIP-pretrained-embeddings/artemis-text/embeddings-clean-artemis'
)
X = numpy.array([
    numpy.load(embd_folder / (str(i) + '.npy')) for i in tqdm(artemis.index)])
X.shape

100%|██████████| 429431/429431 [56:20<00:00, 127.04it/s] 


(429431, 512)

## Train shallow models

In [ ]:
def experiment_shallow(X: Tuple[numpy.ndarray, numpy.ndarray],
                       labels: Tuple[numpy.ndarray, numpy.ndarray]):
    """

    Args:
        X: (N, D) data embeddings.
        labels: (N,) ground truth labels.
        kfolds: Number of K-Folds.

    Returns:
        Dict: Folds' Spearman & Overall Score
    """
    k_folds = model_selection.KFold(n_splits=kfolds, shuffle=True,
                                    random_state=1234)
    log = {f'fold_{k+1}': dict() for k in range(kfolds)}
    foldgen = k_folds.split(X, labels)
    for k, fold in enumerate(foldgen):
        train, test = fold
        X_train, X_test = X[train], X[test]
        y_train, y_test = labels[train], labels[test]

        lr = pipeline.make_pipeline(preprocessing.StandardScaler(
            with_mean=True, with_std=True),
            linear_model.LinearRegression()
        )

        lr.fit(X_train, y_train)
        preds = lr.predict(X_test)
        log[f'fold_{k+1}'] = metrics.compute(y_test, preds)
    return log